# Data preprocess

In [ ]:
import numpy as np
import re
import pandas as pd
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
files = open("./drive/My Drive/par_corp.csv")

re_lines = []
for line in files:
    if line[0] == '[':
        continue
    re_line = re.sub('[#".?!\n]', '', line)
    re_lines.append(re_line)

kor = []
eng = []
count = 0
for line in re_lines:
    count = count + 1
    if count % 2 == 0:
        kor.append(line)
    else:
        eng.append(line)

d = {'kor':kor, 'eng':eng}
par_corp = pd.DataFrame(d)

print(par_corp)

In [ ]:
encoder_input, decoder_input, decoder_output = [], [], []

# '나는 개와 산책을 하고 있다'
# ######## 위 문장의 셀 상태랑 은닉 상태 + <start> 가 인풋으로 들어가면
# '<start> i am taking a walk with my dog' -> 각 시점마다 이 문장의 일부분을 decoder_output을 추측하는데 사용하고 있음
# 'i am taking a walk with my dog <end>'

for stc in par_corp['kor']:
    encoder_input.append(stc.split())

# 스타트 뒤에 띄어쓰기 있습니다
for stc in par_corp['eng']:
    decoder_input.append(("<start> "+stc).split())

# 엔드 앞에 띄어쓰기 있습니다
for stc in par_corp['eng']:
    decoder_output.append((stc+" <end>").split())

In [ ]:
tokenizer_ko = Tokenizer()
tokenizer_ko.fit_on_texts(encoder_input)
encoder_input = tokenizer_ko.texts_to_sequences(encoder_input)

# 만약에 5000이면, 1~4999(패딩하기 전) -> 0~4999(패딩하고 난 뒤)
tokenizer_en = Tokenizer()
tokenizer_en.fit_on_texts(decoder_input)
tokenizer_en.fit_on_texts(decoder_output)
decoder_input = tokenizer_en.texts_to_sequences(decoder_input)
decoder_output = tokenizer_en.texts_to_sequences(decoder_output)

In [ ]:
# 문장 길이 체크
import matplotlib.pyplot as plt

len_ko = []
for data in encoder_input:
    len_ko.append(len(data))

len_en = []
for data in decoder_input:
    len_en.append(len(data))

plt.hist(len_ko, label='ko', alpha=0.7)
plt.hist(len_en, label='en', alpha=0.7)
plt.legend()
plt.show()

In [ ]:
# maxlen 없어도 알아서 잘 패딩합니다
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_output = pad_sequences(decoder_output, padding="post")

In [ ]:
print(encoder_input.shape)
print(decoder_input.shape)

In [ ]:
# 나중에 prediction 할때 사용하기 위함 (인덱스로 단어 찾기)
en_to_index = tokenizer_en.word_index
index_to_en = tokenizer_en.index_word

In [ ]:
test_size = 12000
encoder_input_train = encoder_input[:-test_size]
decoder_input_train = decoder_input[:-test_size]
decoder_output_train = decoder_output[:-test_size]

encoder_input_test = encoder_input[-test_size:]
decoder_input_test = decoder_input[-test_size:]
decoder_output_test = decoder_output[-test_size:]

# Training

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [ ]:
# 인코더 - 한글 문장 받아서 LSTM 마지막 시점의 은닉상태/셀상태 리턴하도록
# 원래는 데이터 갯수랑 문장 길이 같이 들어가야함
# 왜 데이터 갯수는 명시하지 않을까요?
# fit 할때 validation data -> test set -> 데이터 갯수 다르기 때문에
encoder_inputs = Input(shape=(27,)) #27은 문장의 길이
# +1을 해서 패딩까지 고려
encoder_embed = Embedding(len(tokenizer_ko.word_index)+1, 50)(encoder_inputs)
# 패딩 값은 필요없는데... (0에 해당하는 임베딩 벡터 제외)
encoder_mask = Masking(mask_value=0)(encoder_embed)
# return state를 쓰면 마지막 은닉 상태, 마지막 은닉 상태, 마지막 셀 상태 값을 리턴
encoder_outputs, h_state, c_state = LSTM(50, return_state=True)(encoder_mask)

In [ ]:
# 디코더 - 위에서 리턴한 상태값이랑, 영어 문장 입력받아서 LSTM의 출력값 리턴하도록
decoder_inputs = Input(shape=(27,))
decoder_embed = Embedding(len(tokenizer_en.word_index)+1, 50)(decoder_inputs)
decoder_mask = Masking(mask_value=0)(decoder_embed)
# return sequences를 쓰면 전체 시점의 은닉 상태 값을 리턴
# 둘 다 쓰면 전체 시점의 은닉 상태(단어갯수만큼)/마지막 은닉 상태/마지막 셀 상태 값을 리턴
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_mask, initial_state=[h_state, c_state])
decoder_dense = Dense(len(tokenizer_en.word_index)+1, activation='softmax')
decoder_softmax_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
# sparse는 라벨이 정수 형태로 제공될 때 사용되는 함수 (그냥 categorical은 원핫 벡터로 라벨이 제공될 때)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
# 레이어 별로 가중치가 학습되는 것임
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_output_train, validation_data = ([encoder_input_test, decoder_input_test], decoder_output_test), batch_size = 128, epochs = 50)